In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd 
import numpy as np 
import json 

In [0]:
data_credits = pd.read_csv('/content/drive/My Drive/predicting_movie_success_train_set/tmdb_5000_credits_train.csv')
data_movies  = pd.read_csv('/content/drive/My Drive/predicting_movie_success_train_set/tmdb_5000_movies_train.csv')

In [4]:
data_movies.isna().sum()

budget                  1004
genres                  1004
homepage                3382
id                      1004
keywords                1004
original_language       1004
original_title          1004
overview                1005
popularity              1004
production_companies    1004
production_countries    1004
release_date            1004
revenue                 1004
runtime                 1005
spoken_languages        1004
status                  1004
tagline                 1479
title                   1004
vote_average            1004
vote_count              1004
dtype: int64

In [0]:
data_movies = data_movies.dropna(how='all')

In [0]:
combined_data=pd.merge(data_movies,data_credits[['movie_id','cast','crew']],left_on='id',right_on='movie_id')

In [0]:
combined_data.to_csv('/content/drive/My Drive/predicting_movie_success_train_set/CombinedData.csv')

#Continue 

In [0]:
combined_data = pd.read_csv('/content/drive/My Drive/predicting_movie_success_train_set/CombinedData.csv')

In [0]:
combined_data=combined_data.drop(['homepage','id','keywords','original_language','original_title','overview','production_countries','release_date','spoken_languages','status','tagline','title','movie_id'],axis=1)

In [10]:
combined_data.corr()['vote_average']

Unnamed: 0     -0.006369
budget          0.059468
popularity      0.303670
revenue         0.232986
runtime         0.413934
vote_average    1.000000
vote_count      0.382702
Name: vote_average, dtype: float64

In [0]:
def transform_dict(combined_data,col,focus):
  d = dict()
  c = dict()
  i=0
  for item in combined_data[col]:
      temp = json.loads(item) 
      for it in temp : 
        if it[focus] not in d : 
          d[it[focus]]=combined_data.iloc[i,combined_data.shape[1]-4]
          c[it[focus]]=1
        else:
          d[it[focus]]+=(combined_data.iloc[i,combined_data.shape[1]-4])
          c[it[focus]]+=1
      i+=1
  final=dict()
  for item in d : 
      final[item]=d[item]/c[item]
  output=[]
  for item in combined_data[col]:
    tempo = json.loads(item)
    adder=0 
    for it in tempo : 
       adder+=final[it[focus]]
    output.append(adder)
  output = np.reshape(output,(combined_data.shape[0],))
  return output

In [0]:
def transform_crew(combined_data,col,focus):
  d = dict()
  c = dict()
  i=0
  for item in combined_data[col]:
      temp = json.loads(item) 
      for it in temp : 
        if it[focus] not in d and (it['department'] == 'Production' or it['department'] == 'Directing'): 
         
          d[it[focus]]=combined_data.iloc[i,combined_data.shape[1]-4]
          c[it[focus]]=1
        elif it[focus] in d and (it['department'] == 'Production' or it['department'] == 'Directing'): 
          d[it[focus]]+=(combined_data.iloc[i,combined_data.shape[1]-4])
          c[it[focus]]+=1
      i+=1
  final=dict()
  for item in d : 
      final[item]=d[item]/c[item]
      
  output=[]
  for item in combined_data[col]:
    tempo = json.loads(item)
    adder=0 
    for it in tempo : 
       if (it['department'] == 'Production' or it['department'] == 'Directing'):
          adder+=final[it[focus]]
    output.append(adder)
  output = np.reshape(output,(combined_data.shape[0],))
  return output

In [0]:
genres =transform_dict(combined_data,'genres','name')
production_companies = transform_dict(combined_data,'production_companies','name')
cast = transform_dict(combined_data,'cast','name')
crew = transform_crew(combined_data,'crew','name')
#Update 
#keywords = transform_dict(combined_data,'keywords','name')
#spoken_languages = transform_dict(combined_data,'spoken_languages','name') 
#production_countries = transform_dict(combined_data,'production_countries','name')


In [17]:
g = dict() 
g['budget']=combined_data['budget']
g['genres']=genres 
g['popularity']=combined_data['popularity']
g['production_companies']=production_companies
g['revenue']=combined_data['revenue']
g['runtime']=combined_data['runtime']
g['cast']=cast 
g['crew']=crew 
g['vote_count']=combined_data['vote_count']
#g['vote weight']=combined_data['vote_count']*combined_data['vote_average']
#g['keywords']=keywords
#g['spoken_languages']=spoken_languages
#g['production_countries']=production_countries
#g['vote_average']=combined_data['vote_average']
final_data = pd.DataFrame(g)
print(final_data)

           budget     genres  popularity  ...         cast        crew  vote_count
0     237000000.0  24.355775  150.437577  ...   576.968136   68.967292     11800.0
1     300000000.0  18.284440  139.082615  ...   225.451795   77.081883      4500.0
2     245000000.0  18.516796  107.376788  ...   528.118231  159.066861      4466.0
3     250000000.0  24.928594  112.312950  ...  1129.055919  205.726358      9106.0
4     260000000.0  18.276386   43.926995  ...   166.637054  127.631002      2124.0
...           ...        ...         ...  ...          ...         ...         ...
3794    4000000.0   6.460011    3.663108  ...    76.891667   42.166667        36.0
3795     650000.0  30.480578   12.496261  ...    85.990476   68.593333       285.0
3796          0.0   6.460011    0.596889  ...    75.033434   10.991667         7.0
3797          0.0   6.460011    2.611364  ...    39.275000  103.025000        31.0
3798    4000000.0  12.417745    3.238466  ...    62.935832   18.600000        37.0

[37

In [0]:
final_data=final_data.dropna()


In [0]:
final_data.to_csv('/content/drive/My Drive/predicting_movie_success_train_set/FinallData.csv')

In [0]:
final_data = pd.read_csv('/content/drive/My Drive/predicting_movie_success_train_set/FinalData.csv')

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#data = final_data[['budget','genres','popularity','production_companies','revenue','runtime','cast','crew','vote_count','keywords','spoken_languages','production_countries']]
data=final_data[['runtime','vote weight','vote_count']]
labels = final_data['vote_average']
print(final_data.corr()['vote_average'])
data = np.array(data)
labels=np.array(labels)
X_train,X_test,y_train,y_test=train_test_split(data,labels,test_size=0.3,random_state=42)

Unnamed: 0             -0.006592
budget                  0.059642
genres                  0.017728
popularity              0.303980
production_companies    0.165474
revenue                 0.233226
runtime                 0.413934
cast                    0.300295
crew                    0.260597
vote_count              0.382992
vote weight             0.414971
vote_average            1.000000
Name: vote_average, dtype: float64


In [30]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.fit_transform(X_test)
clf_poly = LinearRegression().fit(X_train_poly,y_train)
prd = clf_poly.predict(X_test_poly)
print(mean_squared_error(y_test,prd))

0.5301818291183458


In [31]:
clf = LinearRegression().fit(X_train,y_train)
from sklearn.metrics import mean_squared_error
y_hat = clf.predict(X_test)
print(mean_squared_error(y_test,y_hat))

0.6322594295785637


In [0]:
mx=np.max(temp,axis=0)
mn=np.min(temp,axis=0)
genres_dict=dict()
i = 0 
for item in d:
    genres_dict[item]=((temp[i]-mn)/(mx-mn))
    print(item)
    print(genres_dict)
    i+=1

In [0]:
print(combined_data.shape)

(3799, 24)


In [0]:
combined_data=combined_data.drop(['homepage','id','original_language','original_title','overview','release_date','status','tagline','title','movie_id'],axis=1)